In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torchtext
import pickle
import gensim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from fastai.text import *
from helpers import TextTransform, SearchEngineDataset, pad_collate
from fastai.layers import Flatten
import torch.nn.functional as F
import numpy as np
from models import WideResNet, WideResNetEmbedding, WideResNetParallel

In [3]:
avg_pool5 = nn.AdaptiveAvgPool2d((5, 5))
avg_pool6 = nn.AdaptiveAvgPool2d((6, 6))

In [4]:
max_pool5 = nn.AdaptiveMaxPool2d((5, 5))
max_pool6 = nn.AdaptiveMaxPool2d((6, 6))

In [4]:
x_1x1 = torch.Tensor([[[1.]]])

In [5]:
x_1x2 = torch.Tensor([[[1., 1.], [2., 2.]]])

In [6]:
avg_pool5(x_1x1)

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

In [7]:
avg_pool6(x_1x1)

tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]]])

In [8]:
max_pool5(x_1x1)

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

In [9]:
max_pool6(x_1x1)

tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]]])

In [10]:
avg_pool5(x_1x2)

tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.5000, 1.5000, 1.5000, 1.5000, 1.5000],
         [2.0000, 2.0000, 2.0000, 2.0000, 2.0000],
         [2.0000, 2.0000, 2.0000, 2.0000, 2.0000]]])

In [11]:
avg_pool6(x_1x2)

tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]]])

In [12]:
max_pool5(x_1x2)

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]])

In [13]:
max_pool6(x_1x2)

tensor([[[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2.]]])

In [12]:
itos = pickle.load(open('../../data/train_lm_data/itos.pkl', 'rb'))

In [16]:
itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 'the',
 ',',
 'of',
 'and',
 'a',
 'to',
 '1',
 'in',
 'is',
 'for',
 '-',
 ')',
 '(',
 'or',
 'that',
 'are',
 'you',
 'as',
 'it',
 'on',
 ':',
 'with',
 'your',
 'by',
 'from',
 'an',
 'be',
 'can',
 'this',
 'at',
 "'s",
 'have',
 'was',
 '$',
 'not',
 '2',
 '/',
 'if',
 'one',
 'which',
 'will',
 'more',
 'has',
 ';',
 'but',
 'also',
 'i',
 'they',
 'when',
 'most',
 '3',
 'all',
 'may',
 'about',
 'other',
 'than',
 'time',
 'used',
 '’s',
 'there',
 'up',
 'their',
 'do',
 'first',
 'some',
 'its',
 'two',
 'new',
 'use',
 'how',
 'what',
 '4',
 'into',
 'name',
 'we',
 '%',
 'who',
 'people',
 'only',
 'years',
 'per',
 'average',
 "'",
 'so',
 'out',
 'these',
 'between',
 'many',
 'after',
 'blood',
 'such',
 'year',
 'like',
 'water',
 'his',
 'he',
 'any',
 'body',
 'over',
 'number',
 '5',
 'get',
 'day',
 'no',
 'our',
 'called',
 'states',
 'been',
 'state',
 'make',
 'cost',
 'each',

In [19]:
import numpy as np
def loadGloveModel(gloveFile, d_emb):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        elems = line.split()
        vec = [float(n) for n in elems[-d_emb:]]
        word = ' '.join(elems[:-d_emb])
        model[word] = vec
    print ("Done.",len(model)," words loaded!")
    f.close()
    return model

In [20]:
glove = loadGloveModel('../../../../../Pretrained/glove.840B.300d.txt', 300)

Loading Glove Model
Done. 2195895  words loaded!


In [22]:
itovecs = list()
zero = np.zeros(300)
for elem in itos:
    if elem in glove:
        itovecs.append(glove[elem])
    else:
        itovecs.append(zero)

In [23]:
itovecs = np.array(itovecs)

In [24]:
itovecs.shape

(60002, 300)

In [28]:
np.save('../../data/train_lm_data/itovecs.npy', itovecs)

In [5]:
class SearchEngineDataset(Dataset):
    def __init__(self, csv_path, cols, transform=None):
        self.query_key = 'query'
        self.passage_key = 'passage'
        self.qid_key = 'qid'
        self.pid_key = 'pid'
        self.label_key = 'label'        
        self.data = pd.read_csv('../../data/data.tsv', sep='\t', names=[self.qid_key,
                                                                        self.query_key, 
                                                                        self.passage_key,
                                                                        self.label_key,
                                                                        self.pid_key])
        self.data = self.data[cols]
        self.data_len = len(self.data)
        self.transform = transform
    
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx, :]
        query = row[self.query_key]
        passage = row[self.passage_key]
        label = row[self.label_key]
        if self.transform:
            query = self.transform(query)
            passage = self.transform(passage)
        return ((query, passage), label)
        

In [3]:
text_transform = TextTransform('../../data/train_lm_data/itos.pkl')

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [5]:
search_dataset = SearchEngineDataset('../../data/data_sample.tsv',
                                    ['query', 'passage', 'label'], 
                                     transform=text_transform.text_to_ints)

In [15]:
tokens = tokenizer.process_text('. what is a corporation?', base_tok)

In [17]:
vocab.numericalize(tokens)

[8, 79, 17, 13, 2040, 124]

In [3]:
text_transform.text_to_ints('what is a corporation')

[79, 17, 13, 2040]

In [3]:
relu = nn.ReLU()

In [5]:
F.relu(torch.Tensor([1., 3., -1.5, 7.8]))

tensor([1.0000, 3.0000, 0.0000, 7.8000])

In [6]:
torch.zeros(6, 18)

[autoreload of helpers failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/local/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/local/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/media/saqib/ni/Projects/Microsoft/AI_Challenge_18/code/nbs/helpers.py", li

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
a = [[1, 1], [2, 2], [3, 3]]
b = [[4, 4, 4], [5, 5, 5], [6, 6, 6]]
c = [7, 8, 9]

In [11]:
list(zip(list(zip(a, b)), c))

[(([1, 1], [4, 4, 4]), 7), (([2, 2], [5, 5, 5]), 8), (([3, 3], [6, 6, 6]), 9)]

In [8]:
dl = DataLoader(search_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=pad_collate)

In [181]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, pretrained_wts_pth, emb_dim=300):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        # Intitialize embedding to GloVe weights
        self.emb.weight.data.copy_(torch.from_numpy(np.load(pretrained_wts_pth)))
        self.adap_avg_pool = nn.AdaptiveAvgPool2d((32, 32))
        self.adap_max_pool = nn.AdaptiveMaxPool2d((32, 32))
        
    def forward(self, x):
        q = self.emb(x[0])
        qap = self.adap_avg_pool(q)
        qmp = self.adap_max_pool(q)
        q = torch.cat([qap, qmp], dim=1)

        p = self.emb(x[1]) 
        pap = self.adap_avg_pool(p)
        pmp = self.adap_max_pool(p)
        p = torch.cat([pap, pmp], dim=1)
        
        x = torch.cat([q, p], dim=2)
        x = x.unsqueeze(1)
        return x

In [50]:
wrnp = WideResNetParallel(60002, '../../data/train_lm_data/itovecs.npy', 300, n_grps=2, N=2, k=5)

In [51]:
print(wrnp)

WideResNetParallel(
  (emb): Embedding(60002, 300)
  (adap_avg_pool): AdaptiveAvgPool2d(output_size=(32, 32))
  (adap_max_pool): AdaptiveMaxPool2d(output_size=(32, 32))
  (qwrn): WideResNetOpen(
    (features): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(16, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (drop): Dropout(p=0.3, inplace)
        (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (shortcut): Sequential(
          (0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU(inplace)
          (2): Conv2d(16, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fals

In [52]:
sum(p.numel() for p in wrnp.parameters() if p.requires_grad)

20243297

In [182]:
emb_model = EmbeddingModel(60002, '../../data/train_lm_data/itovecs.npy', emb_dim=300)

In [38]:
x, y = next(iter(dl))
out = wrnp(*x)

In [39]:
out.shape

torch.Size([2, 1])

In [40]:
out

tensor([[0.5234],
        [0.5236]], grad_fn=<SigmoidBackward>)

In [185]:
conv = nn.Conv2d(1, 2, 3)

In [186]:
conv(out)

tensor([[[[-0.1640, -0.1776, -0.0973,  ..., -0.1322, -0.1137, -0.1232],
          [-0.1640, -0.1776, -0.0973,  ..., -0.1121, -0.1115, -0.0635],
          [-0.1734, -0.1580, -0.0890,  ..., -0.1330, -0.1117, -0.1017],
          ...,
          [-0.9917, -1.3308, -0.3863,  ..., -0.1118, -0.1118, -0.1118],
          [-0.1118, -0.1118, -0.1118,  ..., -0.1118, -0.1118, -0.1118],
          [-0.1118, -0.1118, -0.1118,  ..., -0.1118, -0.1118, -0.1118]],

         [[ 0.4030,  0.2226,  0.3714,  ...,  0.3342,  0.3283,  0.2080],
          [ 0.4030,  0.2226,  0.3714,  ...,  0.3437,  0.3010,  0.2197],
          [ 0.4057,  0.2317,  0.3788,  ...,  0.2990,  0.2612,  0.2679],
          ...,
          [ 1.3058, -0.0430,  0.3647,  ...,  0.2921,  0.2921,  0.2921],
          [ 0.2921,  0.2921,  0.2921,  ...,  0.2921,  0.2921,  0.2921],
          [ 0.2921,  0.2921,  0.2921,  ...,  0.2921,  0.2921,  0.2921]]],


        [[[-0.1455, -0.1613, -0.0935,  ..., -0.0876, -0.1411, -0.1232],
          [-0.1455, -0.1613,

In [197]:
wrn = WideResNet(n_grps=3, N=4, k=10)

In [198]:
print(wrn)

WideResNet(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (drop): Dropout(p=0.3, inplace)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential(
        (0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace)
        (2): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (2): BasicBlock(
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), pad

In [199]:
wrn(out)

tensor([[0.4598],
        [0.4578]], grad_fn=<SigmoidBackward>)

In [6]:
next(iter(dl))

ValueError: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/media/saqib/ni/Projects/Microsoft/AI_Challenge_18/code/nbs/helpers.py", line 59, in pad_collate
    targets = torch.FloatTensor([item[1] for item in batch])
ValueError: too many dimensions 'str'


In [9]:
blah = (x[0].cuda(), x[1].cuda())

In [7]:
model = WideResNetEmbedding(60002, '/media/saqib/ni/Projects/Microsoft/AI_Challenge_18/data/train_lm_data/itovecs.npy', 300, 3, 3, 5 
                           )

In [ ]:
x